In [1]:
import math

# Helper function to convert R/N lists to 1/0 lists
def rn_to_binary(results_list):
  """Converts a list of 'R'/'N' strings to a list of 1/0 integers."""
  return [1 if item == 'R' else 0 for item in results_list]

# --- Metric Calculation Functions ---

def calculate_ap(results_binary, total_relevant):
  """Calculates Average Precision (AP) for a single query."""
  if total_relevant == 0:
    return 0.0

  precision_sum = 0.0
  relevant_count = 0
  for k, is_relevant in enumerate(results_binary):
    if is_relevant:
      relevant_count += 1
      precision_at_k = relevant_count / (k + 1)
      precision_sum += precision_at_k

  # Assume 0 precision for any relevant docs not retrieved (though not needed for these examples explicitly)
  return precision_sum / total_relevant

def calculate_r_precision(results_binary, total_relevant):
  """Calculates R-Precision."""
  if total_relevant == 0:
      return 0.0
  if len(results_binary) < total_relevant:
      # Handle cases where fewer docs are returned than R (not in these examples)
      # This might require padding or specific handling based on definition used
      print(f"Warning: Fewer results ({len(results_binary)}) than R ({total_relevant}). R-Prec might be ill-defined.")
      # Assuming we calculate precision based on available results up to R or len(results), let's use min
      limit = min(len(results_binary), total_relevant)
      if limit == 0: return 0.0
      relevant_in_top_r = sum(results_binary[:limit])
      return relevant_in_top_r / limit # Adjust definition if needed
  else:
      relevant_in_top_r = sum(results_binary[:total_relevant])
      return relevant_in_top_r / total_relevant


def calculate_precision_recall_f1(retrieved_count, relevant_found_count, total_relevant_in_collection):
    """Calculates Precision, Recall, and F1 given counts."""
    if retrieved_count == 0:
        precision = 0.0
    else:
        precision = relevant_found_count / retrieved_count

    if total_relevant_in_collection == 0:
        recall = 0.0 # Or undefined, depending on context
    else:
        recall = relevant_found_count / total_relevant_in_collection

    if precision + recall == 0:
        f1 = 0.0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1

def calculate_uninterpolated_precision_at_recall(results_binary, total_relevant, target_recall_level):
    """Calculates uninterpolated precision at a specific recall level."""
    if total_relevant == 0:
        return 0.0

    target_relevant_count = math.ceil(target_recall_level * total_relevant)
    if target_relevant_count == 0: # Handle 0% recall if needed
        return 1.0 # Often defined as 1 at 0 recall, or handle as edge case

    relevant_count = 0
    for k, is_relevant in enumerate(results_binary):
        if is_relevant:
            relevant_count += 1
            if relevant_count >= target_relevant_count:
                 # Found the first point at or past the target recall
                 precision_at_k = relevant_count / (k + 1)
                 return precision_at_k
    # If target recall is never reached
    return 0.0

def calculate_interpolated_precision_at_recall(results_binary, total_relevant, target_recall_level):
    """Calculates interpolated precision at a specific recall level."""
    if total_relevant == 0:
        return 0.0

    precisions = []
    recalls = []
    relevant_count = 0
    for k, is_relevant in enumerate(results_binary):
        if is_relevant:
            relevant_count += 1
            precision_at_k = relevant_count / (k + 1)
            recall_at_k = relevant_count / total_relevant
            precisions.append(precision_at_k)
            recalls.append(recall_at_k)

    max_precision = 0.0
    for p, r in zip(precisions, recalls):
        if r >= target_recall_level:
            max_precision = max(max_precision, p)

    return max_precision

def calculate_kappa(judgments):
    """Calculates Cohen's Kappa for inter-judge agreement.
    Args:
        judgments: A list of tuples, where each tuple is (judge1_score, judge2_score).
                   Scores are assumed to be 0 (non-relevant) or 1 (relevant).
    Returns:
        The kappa statistic.
    """
    n = len(judgments)
    if n == 0:
        return 0.0

    # Contingency Table
    agree_r = 0 # Both 1
    agree_n = 0 # Both 0
    j1r_j2n = 0 # Judge 1 = 1, Judge 2 = 0
    j1n_j2r = 0 # Judge 1 = 0, Judge 2 = 1

    # Marginal counts
    j1_r_count = 0
    j1_n_count = 0
    j2_r_count = 0
    j2_n_count = 0

    for j1, j2 in judgments:
        if j1 == 1 and j2 == 1:
            agree_r += 1
        elif j1 == 0 and j2 == 0:
            agree_n += 1
        elif j1 == 1 and j2 == 0:
            j1r_j2n += 1
        elif j1 == 0 and j2 == 1:
            j1n_j2r += 1

        if j1 == 1: j1_r_count += 1
        else: j1_n_count += 1
        if j2 == 1: j2_r_count += 1
        else: j2_n_count += 1

    # Observed Agreement (Po)
    po = (agree_r + agree_n) / n

    # Expected Agreement (Pe)
    p_j1_r = j1_r_count / n
    p_j1_n = j1_n_count / n
    p_j2_r = j2_r_count / n
    p_j2_n = j2_n_count / n
    pe = (p_j1_r * p_j2_r) + (p_j1_n * p_j2_n)

    # Kappa
    if 1 - pe == 0: # Avoid division by zero if perfect agreement is expected by chance
        return 1.0 if po == 1.0 else 0.0 # Or handle as undefined/special case

    kappa = (po - pe) / (1 - pe)
    return kappa

# --- Exercise 8.8 ---
print("--- Exercise 8.8 ---")
results_s1_rn = ['R', 'N', 'R', 'N', 'N', 'N', 'N', 'N', 'R', 'R']
results_s2_rn = ['N', 'R', 'N', 'N', 'R', 'R', 'R', 'N', 'N', 'N']
total_relevant_8_8 = 4

results_s1_bin = rn_to_binary(results_s1_rn)
results_s2_bin = rn_to_binary(results_s2_rn)

# a. MAP
ap_s1 = calculate_ap(results_s1_bin, total_relevant_8_8)
ap_s2 = calculate_ap(results_s2_bin, total_relevant_8_8)
# Since there's only one "query" per system result, AP = MAP for that system
print(f"a. MAP System 1: {ap_s1:.2f}")
print(f"   MAP System 2: {ap_s2:.2f}")
print(f"   System with higher MAP: {'System 1' if ap_s1 > ap_s2 else 'System 2'}")

# b. Intuition (Explanation is qualitative)
print("\nb. Intuitive Sense: Yes, MAP rewards early retrieval of relevant docs.")
print("   System 1 found relevant docs at ranks 1 and 3, boosting its score early.")
print("   System 2 found relevant docs starting later (rank 2, 5, 6, 7).")

# c. R-Precision
r_prec_s1 = calculate_r_precision(results_s1_bin, total_relevant_8_8)
r_prec_s2 = calculate_r_precision(results_s2_bin, total_relevant_8_8)
print(f"\nc. R-Precision System 1 (R=4): {r_prec_s1:.2f}")
print(f"   R-Precision System 2 (R=4): {r_prec_s2:.2f}")
print(f"   Ranking based on R-Precision ({'System 1 > System 2' if r_prec_s1 > r_prec_s2 else 'System 2 > System 1'}) is the same as MAP.")

# --- Exercise 8.9 ---
print("\n--- Exercise 8.9 ---")
results_8_9_rn = ['R', 'R', 'N', 'N', 'N', 'N', 'N', 'N', 'R', 'N',
                  'R', 'N', 'N', 'N', 'R', 'N', 'N', 'N', 'N', 'R']
total_relevant_8_9 = 8
results_8_9_bin = rn_to_binary(results_8_9_rn)
retrieved_count_8_9 = 20
relevant_found_8_9 = sum(results_8_9_bin)

# a. Precision at top 20
precision_at_20, _, _ = calculate_precision_recall_f1(retrieved_count_8_9, relevant_found_8_9, total_relevant_8_9)
print(f"a. Precision@20: {precision_at_20:.2f}")

# b. F1 at top 20
_, _, f1_at_20 = calculate_precision_recall_f1(retrieved_count_8_9, relevant_found_8_9, total_relevant_8_9)
print(f"b. F1@20: {f1_at_20:.4f}")

# c. Uninterpolated Precision at 25% Recall
recall_target_c = 0.25
uninterp_prec_at_25_recall = calculate_uninterpolated_precision_at_recall(results_8_9_bin, total_relevant_8_9, recall_target_c)
print(f"c. Uninterpolated Precision at {recall_target_c*100}% Recall: {uninterp_prec_at_25_recall:.2f}")

# d. Interpolated Precision at 33% Recall
recall_target_d = 0.33
interp_prec_at_33_recall = calculate_interpolated_precision_at_recall(results_8_9_bin, total_relevant_8_9, recall_target_d)
print(f"d. Interpolated Precision at {recall_target_d*100:.0f}% Recall: {interp_prec_at_33_recall:.2f}") # Match report format

# e. MAP for the query (assuming top 20 is complete set)
# Note: AP calculation uses total_relevant_in_collection, but for this specific question,
# the report calculates AP based only on the 6 found relevant docs. Let's recalculate AP
# considering only the 6 found ones as the denominator as per the report's method for this point.
# However, the standard AP definition uses total_relevant_in_collection (8).
# Let's use the standard definition first, then match the report.

# Standard AP definition (denominator = 8)
ap_8_9_std = calculate_ap(results_8_9_bin, total_relevant_8_9)
print(f"e. MAP (Standard AP, denominator=8): {ap_8_9_std:.4f}")

# AP as calculated in the report (denominator = 6, only considering found relevant docs)
# Re-implementing the specific summation from the report for clarity
precisions_at_relevant_ranks_8_9e = [1/1, 2/2, 3/9, 4/11, 5/15, 6/20]
ap_8_9_report = sum(precisions_at_relevant_ranks_8_9e) / relevant_found_8_9 # Divide by 6 found
print(f"   MAP (Report's method for point e, denominator=6): {ap_8_9_report:.4f}")

# f. Largest possible MAP
# Achieved when all 8 relevant docs are at ranks 1-8
map_max_8_9 = 1.0
print(f"f. Largest possible MAP: {map_max_8_9:.1f}")

# g. Smallest possible MAP
# Achieved when all 8 relevant docs are at ranks 9993-10000 in a 10k collection
# Calculation from report:
precisions_min = [i / (9992 + i) for i in range(1, 9)] # Approximation
map_min_8_9_approx = sum(precisions_min) / 8
map_min_8_9_report = 0.00045 # From report
print(f"g. Smallest possible MAP (from report): {map_min_8_9_report:.5f}")
# print(f"   (Approximation: {map_min_8_9_approx:.5f})") # Show calculation if needed

# h. Largest error
map_e = ap_8_9_report # Using the report's value for consistency
error_vs_max = abs(map_e - map_max_8_9)
error_vs_min = abs(map_e - map_min_8_9_report)
max_error = max(error_vs_max, error_vs_min)
print(f"h. Error vs Max MAP: |{map_e:.4f} - {map_max_8_9:.1f}| = {error_vs_max:.4f}")
print(f"   Error vs Min MAP: |{map_e:.4f} - {map_min_8_9_report:.5f}| = {error_vs_min:.4f}")
print(f"   Largest absolute error: {max_error:.4f} (or {max_error*100:.1f}%)")

# --- Exercise 8.10 ---
print("\n--- Exercise 8.10 ---")
# Judgments: (docID, judge1, judge2) - Storing as list of tuples for kappa function
judgments_list = [
    (1, 0, 0), (2, 0, 0), (3, 1, 1), (4, 1, 1), (5, 1, 0), (6, 1, 0),
    (7, 1, 0), (8, 1, 0), (9, 0, 1), (10, 0, 1), (11, 0, 1), (12, 0, 1)
]
# Store judgments in a dict for easier lookup by docID if needed later
judgments_dict = {doc_id: (j1, j2) for doc_id, j1, j2 in judgments_list}

retrieved_docs_8_10 = {4, 5, 6, 7, 8}
retrieved_count_8_10 = len(retrieved_docs_8_10)

# a. Kappa Measure
kappa_score = calculate_kappa([(j1, j2) for _, j1, j2 in judgments_list])
print(f"a. Kappa measure: {kappa_score:.3f}")

# b. P, R, F1 (Relevant = Both judges agree)
relevant_set_b = {doc_id for doc_id, j1, j2 in judgments_list if j1 == 1 and j2 == 1} # {3, 4}
total_relevant_b = len(relevant_set_b)
relevant_found_b = len(retrieved_docs_8_10.intersection(relevant_set_b)) # {4} -> count = 1

precision_b, recall_b, f1_b = calculate_precision_recall_f1(retrieved_count_8_10, relevant_found_b, total_relevant_b)
print(f"\nb. Relevant if both judges agree:")
print(f"   Relevant Set: {relevant_set_b}")
print(f"   Retrieved & Relevant: {retrieved_docs_8_10.intersection(relevant_set_b)}")
print(f"   Precision: {relevant_found_b}/{retrieved_count_8_10} = {precision_b:.3f}")
print(f"   Recall:    {relevant_found_b}/{total_relevant_b} = {recall_b:.3f}")
print(f"   F1:        {f1_b:.3f}")

# c. P, R, F1 (Relevant = Either judge agrees)
relevant_set_c = {doc_id for doc_id, j1, j2 in judgments_list if j1 == 1 or j2 == 1} # {3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
total_relevant_c = len(relevant_set_c)
relevant_found_c = len(retrieved_docs_8_10.intersection(relevant_set_c)) # {4, 5, 6, 7, 8} -> count = 5

precision_c, recall_c, f1_c = calculate_precision_recall_f1(retrieved_count_8_10, relevant_found_c, total_relevant_c)
print(f"\nc. Relevant if either judge agrees:")
print(f"   Relevant Set (Size): {total_relevant_c}") # Print size for brevity
print(f"   Retrieved & Relevant: {retrieved_docs_8_10.intersection(relevant_set_c)}")
print(f"   Precision: {relevant_found_c}/{retrieved_count_8_10} = {precision_c:.3f}")
print(f"   Recall:    {relevant_found_c}/{total_relevant_c} = {recall_c:.3f}")
print(f"   F1:        {f1_c:.3f}")

--- Exercise 8.8 ---
a. MAP System 1: 0.60
   MAP System 2: 0.49
   System with higher MAP: System 1

b. Intuitive Sense: Yes, MAP rewards early retrieval of relevant docs.
   System 1 found relevant docs at ranks 1 and 3, boosting its score early.
   System 2 found relevant docs starting later (rank 2, 5, 6, 7).

c. R-Precision System 1 (R=4): 0.50
   R-Precision System 2 (R=4): 0.25
   Ranking based on R-Precision (System 1 > System 2) is the same as MAP.

--- Exercise 8.9 ---
a. Precision@20: 0.30
b. F1@20: 0.4286
c. Uninterpolated Precision at 25.0% Recall: 1.00
d. Interpolated Precision at 33% Recall: 0.36
e. MAP (Standard AP, denominator=8): 0.4163
   MAP (Report's method for point e, denominator=6): 0.5551
f. Largest possible MAP: 1.0
g. Smallest possible MAP (from report): 0.00045
h. Error vs Max MAP: |0.5551 - 1.0| = 0.4449
   Error vs Min MAP: |0.5551 - 0.00045| = 0.5546
   Largest absolute error: 0.5546 (or 55.5%)

--- Exercise 8.10 ---
a. Kappa measure: -0.333

b. Relevant 